In [2]:
import pandas as pd
pd.options.display.max_columns=200
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.pipeline import make_pipeline

In [4]:
data = pd.read_csv('QueryResults.csv')

In [5]:
data

,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount
0,What's a good C decompiler?,<p>I am searching for a decompiler for a C pro...,<c><linux><reverse-engineering><decompiling><a...,193896,89,187350,32,5
